# DCF Part 2: Historical Analysis

In this notebook, we do historical analysis of our target firm. This is all backward looking.

The primary goal is to help us understand the firm's context which in turn helps us understand how to best make projections into the future. This is not the only analysis we need, but we do need it.

The focus will be on key valuation metrics:
* Growth Rates
* Margins
* Reinvestment

## Plan

1. Get Income Statement data: Revenue and EBIT. Compute growth rates in both for as long a history as we can (4 years for Yahoo)
2. Compute EBIT Margin (EBIT/Revenue) and Gross Margin (Gross Profit/Revenue)
3. Get Balance Sheet data to compute NWC and Change in NWC
4. Get Statement of Cash Flows to get Capital Expenditures and Depreciation & Amortization.
5. Compute Reinvestment = CapEx - D&A + Change in NWC
6. Compute FCF, NOPAT and Reinvestment Rate = Reinvestment/NOPAT


This is our in-class project that we will work on progressively through the mod.

Expectations:
1. Notebook is clean and neat, with no repeated code. It has clearly labeled sections for inputs/imports at the beginning. Code is sufficiently commented to demonstrate your understanding of the code and help you or anyone else who may use this code later. All numbers should be formatted so they are readable (so, use commas with large numbers, only a few decimal points).
2. All calculations are correct and all discussion questions are answered completely but concisely, demonstrating a depth of understanding.

Workflow:
1. Notebooks are inherently experimental and allow you to try things, however that requires some good habits
2. Once you are "done" in any sense, you always need to "clean up" your notebook to make it presentable. You'd do the same in Excel - you've tried lots of things, etc. but before you present it, you clean it up.
3. Finally, restart the Runtime/Kernel and run it cleanly all the way top to bottom one time.

Then, its ready to go.


## 1 Always put Imports and Installs at the beginning

And only put them in once.

Set display options here if you are using them.

Set API Keys here if you need them.

In [85]:
# necessary imports
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
from datetime import datetime
from google.colab import userdata

In [86]:
# Format setting for the model - tame the decimals!
pd.options.display.float_format = '{:,.2f}'.format

In [87]:
# Set Ticker we are modeling
ticker_symbol = 'MSFT'

eff_tax_rate = 0.19 # need to set Effective Tax Rate, validate
marg_tax_rate = 0.25

# this is what we divide numbers by, 1000000 is $M
scale_factor = 1000000
scale_name = 'M'

## 2 Get Income Statement Data

In [88]:
# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Get annual income statement data, transpose and sort so numeric index is in correct order.
# OUTPUT should have 4 rows, one for each date and column names across the top are IS fields
income_statement = ticker.financials.T.sort_index()

In [89]:
# lots of data in this one, so print it out with .info()
income_statement.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4 entries, 2022-06-30 to 2025-06-30
Data columns (total 47 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Tax Effect Of Unusual Items                                 4 non-null      float64
 1   Tax Rate For Calcs                                          4 non-null      float64
 2   Normalized EBITDA                                           4 non-null      float64
 3   Total Unusual Items                                         4 non-null      float64
 4   Total Unusual Items Excluding Goodwill                      4 non-null      float64
 5   Net Income From Continuing Operation Net Minority Interest  4 non-null      float64
 6   Reconciled Depreciation                                     4 non-null      float64
 7   Reconciled Cost Of Revenue                                  4 non-null  

In [90]:
# print key columns: Total Revenue, EBIT, Gross Profit
income_statement[['Total Revenue', 'EBIT', 'Gross Profit']]/scale_factor

,Total Revenue,EBIT,Gross Profit
2022-06-30,"198,270.00","85,779.00","135,620.00"
2023-06-30,"211,915.00","91,279.00","146,052.00"
2024-06-30,"245,122.00","110,722.00","171,008.00"
2025-06-30,"281,724.00","126,012.00","193,893.00"


### Compute Growth Rates and Margins

In [91]:
# Calculate Gross Margin
gross_margin = (income_statement['Gross Profit'] / income_statement['Total Revenue'])

# Calculate EBIT Margin
ebit_margin = (income_statement['EBIT'] / income_statement['Total Revenue'])

# Compute Revenue Growth, EBIT Growth
revenue_growth = income_statement['Total Revenue'].pct_change()
ebit_growth = income_statement['EBIT'].pct_change()

# Create a dataframe called df_stats that has the following:
# Revenue Growth, EBIT Growth, Gross Margin, EBIT Margin, Tax Rate for Calcs (eff tax rate) - column names. Rows are dates (year)
# How you do this may vary depending on how you did things above
df_stats = pd.DataFrame({
    'Revenue Growth': revenue_growth,
    'EBIT Growth': ebit_growth,
    'Gross Margin': gross_margin,
    'EBIT Margin': ebit_margin,
    'Tax Rate for Calcs': income_statement['Tax Rate For Calcs']
})
# print it out
display(df_stats)

,Revenue Growth,EBIT Growth,Gross Margin,EBIT Margin,Tax Rate for Calcs
2022-06-30,NaN,NaN,0.68,0.43,0.13
2023-06-30,0.07,0.06,0.69,0.43,0.19
2024-06-30,0.16,0.21,0.70,0.45,0.18
2025-06-30,0.15,0.14,0.69,0.45,0.18


## 3 Get Balance Sheet Data

In [92]:
# Repeat the above but with the balance sheet - get it, transpose so dates are rows, columns are fields
balance_sheet = ticker.balance_sheet.T.sort_index()

In [93]:
# again use .info() so you can see what you have
balance_sheet.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2021-06-30 to 2025-06-30
Data columns (total 78 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Ordinary Shares Number                                  4 non-null      float64
 1   Share Issued                                            4 non-null      float64
 2   Net Debt                                                4 non-null      float64
 3   Total Debt                                              4 non-null      float64
 4   Tangible Book Value                                     4 non-null      float64
 5   Invested Capital                                        4 non-null      float64
 6   Working Capital                                         4 non-null      float64
 7   Net Tangible Assets                                     4 non-null      float64
 8   Capital Lease Obligatio

In [94]:
# print key columns for Non-Cash NWC: Current Assets, Liabilities, Cash, Current Debt
display(balance_sheet[['Current Assets', 'Current Liabilities', 'Cash And Cash Equivalents', 'Current Debt']]/scale_factor)

,Current Assets,Current Liabilities,Cash And Cash Equivalents,Current Debt
2021-06-30,NaN,NaN,NaN,NaN
2022-06-30,"169,684.00","95,082.00","13,931.00","2,749.00"
2023-06-30,"184,257.00","104,149.00","34,704.00","5,247.00"
2024-06-30,"159,734.00","125,286.00","18,315.00","8,942.00"
2025-06-30,"191,131.00","141,218.00","30,242.00","2,999.00"


### Compute NWC and Change in NWC

In [95]:
# Adjust Current Assets by subtracting Cash
adjusted_current_assets = balance_sheet['Current Assets'] - balance_sheet['Cash Cash Equivalents And Short Term Investments']

# Adjust Current Liabilities by subtracting Current Debt
adjusted_current_liabilities = balance_sheet['Current Liabilities'] - balance_sheet['Current Debt And Capital Lease Obligation']

# Calculate NWC = Adj CA - Adj CL
nwc = adjusted_current_assets - adjusted_current_liabilities

# Calculate the Change in NWC
change_in_nwc = nwc.diff()

# Create a new DataFrame with the required column
nwc_change_df = pd.DataFrame({'NWC': nwc,'Change in NWC': change_in_nwc})

# Display the new DataFrame
display(nwc_change_df/scale_factor)

,NWC,Change in NWC
2021-06-30,NaN,NaN
2022-06-30,"-27,398.00",NaN
2023-06-30,"-25,901.00","1,497.00"
2024-06-30,"-32,141.00","-6,240.00"
2025-06-30,"-41,643.00","-9,502.00"


## 4 Get Statement of Cash Flows

In [96]:
# Repeat the above but with SCF
cash_flows = ticker.cashflow.T.sort_index()

In [97]:
# print the .info()
cash_flows.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2021-06-30 to 2025-06-30
Data columns (total 59 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Free Cash Flow                                  4 non-null      float64
 1   Repurchase Of Capital Stock                     4 non-null      float64
 2   Repayment Of Debt                               4 non-null      float64
 3   Issuance Of Debt                                4 non-null      float64
 4   Issuance Of Capital Stock                       4 non-null      float64
 5   Capital Expenditure                             4 non-null      float64
 6   End Cash Position                               4 non-null      float64
 7   Beginning Cash Position                         4 non-null      float64
 8   Effect Of Exchange Rate Changes                 4 non-null      float64
 9   Changes In Cash           

In [98]:
# print key columns - CapEx and D&A
display(cash_flows[['Capital Expenditure', 'Depreciation And Amortization']]/scale_factor)

,Capital Expenditure,Depreciation And Amortization
2021-06-30,NaN,NaN
2022-06-30,"-23,886.00","14,460.00"
2023-06-30,"-28,107.00","13,861.00"
2024-06-30,"-44,477.00","22,287.00"
2025-06-30,"-64,551.00","34,153.00"


In [109]:
# merge cash_flows and nwc_change_df on the date index
merged_df = cash_flows.join(nwc_change_df)

# Display the merged DataFrame, showing CapEx, D&A and Ch in NWC
display(merged_df[['Capital Expenditure', 'Depreciation And Amortization', 'Change in NWC']]/scale_factor)

,Capital Expenditure,Depreciation And Amortization,Change in NWC
2021-06-30,NaN,NaN,NaN
2022-06-30,"-23,886.00","14,460.00",NaN
2023-06-30,"-28,107.00","13,861.00","1,497.00"
2024-06-30,"-44,477.00","22,287.00","-6,240.00"
2025-06-30,"-64,551.00","34,153.00","-9,502.00"


### Compute Reinvestment and NOPAT

In [111]:
# Reverse the sign of the 'Capital Expenditure' to make it positive (my personal preference - optional)
merged_df['Capital Expenditure'] = -merged_df['Capital Expenditure']

# Calculate Reinvestment = CapEx - D&A + Ch in NWC
merged_df['Reinvestment'] = merged_df['Capital Expenditure']

# Extract the 'Reinvestment' column to a new DataFrame
reinvestment_df = merged_df[['Reinvestment']]/scale_factor

# Display the 'Reinvestment' DataFrame
reinvestment_df

,Reinvestment
2021-06-30,NaN
2022-06-30,"-23,886.00"
2023-06-30,"-28,107.00"
2024-06-30,"-44,477.00"
2025-06-30,"-64,551.00"


In [101]:
# Calculate NOPAT in the income statement dataframe
income_statement['NOPAT'] = income_statement['EBIT'] * (1 - income_statement['Tax Rate For Calcs'])

# Extract the 'NOPAT' column to a new DataFrame
nopat_df = income_statement[['NOPAT']]

# Display the 'NOPAT' DataFrame
display(nopat_df/scale_factor)

,NOPAT
2022-06-30,"74,541.95"
2023-06-30,"73,935.99"
2024-06-30,"90,570.60"
2025-06-30,"103,833.89"


## 5 Compute Reinvestment Rate

In [102]:
# Calculate Reinvestment Rate = Reinvestment/NOPAT

# merge in nopat_df and reinvestment_df (I kept it all in nopat_df)
reinvestment_rate_df = nopat_df.merge(reinvestment_df, left_index=True, right_index=True)

# compute reinvestment rate
reinvestment_rate_df['Reinvestment Rate'] = reinvestment_rate_df['Reinvestment'] / reinvestment_rate_df['NOPAT']

# Now, let's display the updated DataFrame with all the required information
display(reinvestment_rate_df.style.format({
    'NOPAT': lambda x: f'{x/scale_factor:,.2f}',
    'Reinvestment': lambda x: f'{x/scale_factor:,.2f}',
    'Reinvestment Rate': '{:.2%}'.format
}))

,NOPAT,Reinvestment,Reinvestment Rate
2022-06-30 00:00:00,"74,541.95",nan,nan%
2023-06-30 00:00:00,"73,935.99","15,743.00",21.29%
2024-06-30 00:00:00,"90,570.60","15,950.00",17.61%
2025-06-30 00:00:00,"103,833.89","20,896.00",20.12%


## 6 Final Result

In [103]:
# now merge in the new Reinvestment Rate column into df_stats
df_stats = df_stats.merge(reinvestment_rate_df['Reinvestment Rate'], left_index=True, right_index=True)

df_stats

,Revenue Growth,EBIT Growth,Gross Margin,EBIT Margin,Tax Rate for Calcs,Reinvestment Rate
2022-06-30,NaN,NaN,0.68,0.43,0.13,NaN
2023-06-30,0.07,0.06,0.69,0.43,0.19,0.21
2024-06-30,0.16,0.21,0.70,0.45,0.18,0.18
2025-06-30,0.15,0.14,0.69,0.45,0.18,0.20


## Homework

Compute EBITDA Margin

Output is a dataframe with a column called "EBITDA Margin" (there can be other columns also, but be sure to show this one by itself)

In [104]:
# Compute EBITDA Margin
ebitda_margin = income_statement['EBITDA'] / income_statement['Total Revenue']

# Create a DataFrame with the EBITDA Margin
ebitda_margin_df = pd.DataFrame({'EBITDA Margin': ebitda_margin})

# Display the DataFrame
display(ebitda_margin_df)

,EBITDA Margin
2022-06-30,0.51
2023-06-30,0.50
2024-06-30,0.54
2025-06-30,0.57
